In [1]:
import dlt
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import fsspec
from aiohttp import ClientTimeout

In [2]:
@dlt.resource(table_name="comments_2022")
def my_df():
    parquet_url = (
        "https://datasets-documentation.s3.eu-west-3.amazonaws.com/"
        "stackoverflow/parquet/comments/2022.parquet"
    )

    timeout = ClientTimeout(total=600)
    fs = fsspec.filesystem("https", client_kwargs={"timeout": timeout})

    dataset = ds.dataset(
        parquet_url,
        format="parquet",
        filesystem=fs,
        partitioning="hive"
    )

    for batch in dataset.to_batches(batch_size=50000):
        df = batch.to_pandas()
        yield df


/opt/conda/lib/python3.11/site-packages/dlt/common/configuration/specs/config_providers_context.py:138 DeprecatedImportWarning: The `airflow.operators.python.get_current_context` attribute is deprecated. Please use `'airflow.sdk.get_current_context'`.

In [3]:
pipeline = dlt.pipeline(
    pipeline_name="parquet_to_minio",
    destination="filesystem",
    dataset_name="comments",
)

load_info = pipeline.run(
    my_df,
    loader_file_format="parquet",
    write_disposition="replace"
)
print(load_info)

/opt/conda/lib/python3.11/site-packages/dlt/common/runtime/exec_info.py:89 DeprecatedImportWarning: The `airflow.operators.python.get_current_context` attribute is deprecated. Please use `'airflow.sdk.get_current_context'`.

Pipeline parquet_to_minio load step completed in 15.42 seconds
1 load package(s) were loaded to destination filesystem and into dataset comments
The filesystem destination used s3://bronze location to store data
Load package 1759958974.4578214 is LOADED and contains no failed jobs
